# Reconocimiento facial
## Problemas:
Se tiene problemas para instalar en entornos linux y windows la libreria face-recognition, para que funcione en windows....

In [2]:
!pip install face-recognition
!pip install opencv-contrib-python

In [8]:
!pip install tabulate

## 1. Extraer rostros

In [3]:
import cv2
import os
imagesPath = "images"
if not os.path.exists("faces"):
    os.makedirs("faces")
    print("Nueva carpeta: faces")
# Detector facial
faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
count = 0
for imageName in os.listdir(imagesPath):
    print(imageName)
    image = cv2.imread(imagesPath + "/" + imageName)
    faces = faceClassif.detectMultiScale(image, 1.1, 5)
    for (x, y, w, h) in faces:
        #cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        face = image[y:y + h, x:x + w]
        face = cv2.resize(face, (150, 150))
        cv2.imwrite("faces/" + str(count) + ".jpg", face)
        count += 1
        cv2.imshow("face", face)
        cv2.waitKey(0)
    cv2.imshow("Image", image)
    cv2.waitKey(0)
cv2.destroyAllWindows()

Nueva carpeta: faces
foto.png
foto3.jpg


In [9]:
import cv2
import os
import face_recognition
from datetime import datetime
from tabulate import tabulate

# Codificar los rostros extraidos
imageFacesPath = "faces"
facesEncodings = []
facesNames = []
asistencia = []


def registrar_asistencia(nombre_estudiante):
    # Verificar si el estudiante ya está registrado
    for registro in asistencia:
        if registro['nombre'] == nombre_estudiante:
            print(f"{nombre_estudiante} ya está registrado.")
            return
    
    # Obtener la hora actual
    hora_registro = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Crear el registro
    registro = {'nombre': nombre_estudiante, 'hora': hora_registro}
    
    # Añadir el registro a la lista de asistencia
    asistencia.append(registro)
    
    # Confirmar el registro
    print(f"Registro exitoso: {nombre_estudiante} a las {hora_registro}")

def reporteAsistencia():
    tabla_asistencia = [[registro['nombre'], registro['hora']] for registro in asistencia]

    # Imprimir la tabla de asistencia
    print("\nRegistros de asistencia:")
    print(tabulate(tabla_asistencia, headers=["Nombre del Estudiante", "Hora de Registro"]))    



for file_name in os.listdir(imageFacesPath):
    image = cv2.imread(imageFacesPath + "/" + file_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    f_coding = face_recognition.face_encodings(image, known_face_locations=[(0, 150, 150, 0)])[0]
    facesEncodings.append(f_coding)
    facesNames.append(file_name.split(".")[0])
print(facesEncodings)
print(facesNames)
##############################################
# LEYENDO VIDEO
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
# Detector facial
faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
while True:
    ret, frame = cap.read()
    if ret == False:
        break
    frame = cv2.flip(frame, 1)
    orig = frame.copy()
    faces = faceClassif.detectMultiScale(frame, 1.1, 5)
    for (x, y, w, h) in faces:
        face = orig[y:y + h, x:x + w]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        actual_face_encoding = face_recognition.face_encodings(face, known_face_locations=[(0, w, h, 0)])[0]
        result = face_recognition.compare_faces(facesEncodings, actual_face_encoding)
        #print(result)
        if True in result:
            
            index = result.index(True)
            name = facesNames[index]
            color = (125, 220, 0)
            print ("Registrar asistencia para "+name )
            registrar_asistencia(name)
        else:
            name = "Desconocido"
            color = (50, 50, 255)
        cv2.rectangle(frame, (x, y + h), (x + w, y + h + 30), color, -1)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, name, (x, y + h + 25), 2, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow("Frame", frame)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()

reporteAsistencia()




[array([-9.15661082e-02,  1.36579022e-01,  7.17975274e-02, -5.62192425e-02,
       -1.21400706e-01,  4.31681983e-02,  1.99564360e-03, -5.38329333e-02,
        1.42262518e-01, -6.27085939e-02,  1.62238896e-01, -1.04938718e-02,
       -2.23905727e-01, -1.18707791e-01,  8.55258554e-02,  1.43808827e-01,
       -1.22028328e-01, -2.00594828e-01, -1.94859505e-01, -2.93053836e-02,
       -3.36377360e-02,  8.05812478e-02,  8.76241131e-04,  3.72676849e-02,
       -1.81978196e-01, -3.60537320e-01, -2.81609595e-03, -1.85565054e-01,
        7.14794733e-03, -1.00516550e-01, -7.27575943e-02,  2.05796510e-02,
       -1.23521328e-01, -6.61925226e-02,  5.97880334e-02,  8.74468759e-02,
       -1.02262311e-01, -8.27184319e-02,  2.41326883e-01,  1.51280733e-02,
       -2.63159722e-01,  4.67597507e-04,  4.81866002e-02,  2.67007947e-01,
        2.64087081e-01, -1.20223789e-02,  1.57486573e-02, -6.28709495e-02,
        1.60285756e-01, -2.45452061e-01,  4.29047607e-02,  1.42339408e-01,
        8.70351121e-02, 